# Limitations

In this tutorial, we show some limitations when using ``pax.Module`` and the rationale behind it.

## No unregistered `ndarray`'s

A PAX's module should not include any unregistered `ndarray`. 
This is to make sure every `ndarray` is a leaf of the pytree.

In [1]:
# uncomment the following line to install pax
# !pip install -q git+https://github.com/NTT123/pax.git

In [2]:
import pax
import jax
import jax.numpy as jnp
import opax

In [3]:
class M1(pax.Module):
    def __init__(self):
        super().__init__()
        self.a = [jnp.array(0.0), jnp.array(1)]


net = M1()

ValueError: Unregistered field `M1.a` (PaxFieldKind.OTHERS) contains a ndarray. Consider registering it using `self.register_*` methods.

Whenever we try to assign a pytree that contains a `ndarray` leaf, PAX will raise a ``ValueError``
informing that we should register it as part of the pytree.

The correct implementation should be:

In [4]:
class FixedM1(pax.Module):
    def __init__(self):
        super().__init__()
        self.register_states("a", [jnp.array(0.0), jnp.array(1)])


net = FixedM1()